In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import io
import os
from scipy.stats import mannwhitneyu

In [2]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

In [4]:
pangenie_svs_sample = read_vcf("pangenie_merged_bi_nosnvs_26sample.vcf")
pangenie_svs_sample[['INFO_ID']] = pangenie_svs_sample['INFO'].str.split(';', expand=True)[2]
pangenie_svs_sample['INFO_ID'] = pangenie_svs_sample['INFO_ID'].str.lstrip('ID=')
pangenie_svs_sample[['type']] = pangenie_svs_sample['INFO_ID'].str.split('-', expand=True)[2]

In [5]:
pangenie_svs_sample.shape

(1129810, 37)

In [6]:
col_list = pangenie_svs_sample['type'].values.tolist()
pd.Series(col_list).value_counts()

DEL    672432
INS    457378
dtype: int64

In [7]:
pangenie_DEL_sample = pangenie_svs_sample[pangenie_svs_sample['INFO_ID'].str.contains('DEL')]
pangenie_INS_sample = pangenie_svs_sample[pangenie_svs_sample['INFO_ID'].str.contains('INS')]

#### deletions

In [8]:
pangenie_DEL_sample_new = pangenie_DEL_sample[[
 'CHROM','FILTER', 'INFO_ID', 'FORMAT', 'HG01114','HG03065','HG01505','HG02492','HG00096','HG03732','HG01573','NA20509','HG02587','HG01596','NA19983','HG00864','HG02011','HG00733','HG03371','NA19036','NA20847','HG00171','NA18534','NA18939','HG03683','NA19240','HG03009','HG02018','HG00514','NA19650'
     ]]
### removed NA12329

In [114]:
for col in pangenie_DEL_sample_new.iloc[:, 4:]:
    pangenie_DEL_sample_new[col] = pangenie_DEL_sample_new[col].str[:3]

In [115]:
for col in pangenie_DEL_sample_new.columns:
    pangenie_DEL_sample_new.loc[pangenie_DEL_sample_new[col].astype(str).str.startswith('0/0'), col] = '0'
    pangenie_DEL_sample_new.loc[pangenie_DEL_sample_new[col].astype(str).str.startswith('.:.'), col] = '0'
    pangenie_DEL_sample_new.loc[pangenie_DEL_sample_new[col].astype(str).str.startswith('0/1'), col] = '1'
    pangenie_DEL_sample_new.loc[pangenie_DEL_sample_new[col].astype(str).str.startswith('1/0'), col] = '1'
    pangenie_DEL_sample_new.loc[pangenie_DEL_sample_new[col].astype(str).str.startswith('1/1'), col] = '1'

In [116]:
pangenie_DEL_sample_new.iloc[:,4: ] = pangenie_DEL_sample_new.iloc[:,4: ].astype(str).astype(int)
pangenie_DEL_sample_new['Sum']=pangenie_DEL_sample_new.iloc[:,4: ].sum(axis=1)
pangenie_DEL_sample_new_atleast3 = pangenie_DEL_sample_new[pangenie_DEL_sample_new['Sum'].astype(int) >= 5]
pangenie_DEL_sample_new_atleast3['START_POS'] = pangenie_DEL_sample_new_atleast3['INFO_ID'].str.split('-', expand=True)[1]
pangenie_DEL_sample_new_atleast3['TYPE'] = pangenie_DEL_sample_new_atleast3['INFO_ID'].str.split('-', expand=True)[2]
pangenie_DEL_sample_new_atleast3['LEN'] = pangenie_DEL_sample_new_atleast3['INFO_ID'].str.split('-', expand=True)[3]

In [117]:
pangenie_DEL_sample_new_atleast3.iloc[:,[31,33]] = pangenie_DEL_sample_new_atleast3.iloc[:,[31,33]].astype(str).astype(int)
# VCF files in fact are providing a single 1-based position for a variant, end = start + len -1
pangenie_DEL_sample_new_atleast3['END_POS'] = pangenie_DEL_sample_new_atleast3['START_POS'] + pangenie_DEL_sample_new_atleast3['LEN']-1
pangenie_DEL_sample_new_atleast3_new = pangenie_DEL_sample_new_atleast3[[
'CHROM', 'START_POS','END_POS', 'TYPE','LEN', 'FILTER', 'INFO_ID', 'FORMAT', 'HG01114','HG03065','HG01505','HG02492','HG00096','HG03732','HG01573','NA20509','HG02587','HG01596','NA19983','HG00864','HG02011','HG00733','HG03371','NA19036','NA20847','HG00171','NA18534','NA18939','HG03683','NA19240','HG03009','HG02018','HG00514','NA19650', 'Sum'
    ]]

In [13]:
pangenie_DEL_sample_new_atleast3_50 = pangenie_DEL_sample_new_atleast3_new[pangenie_DEL_sample_new_atleast3_new['LEN'] >= 50]
pangenie_DEL_sample_new_atleast3_50.shape

(13876, 35)

In [14]:
pangenie_DEL_sample_new_atleast3_50_zero = pangenie_DEL_sample_new_atleast3_50[pangenie_DEL_sample_new_atleast3_50.iloc[:, 8:].eq(0).any(1)]
pangenie_DEL_sample_new_atleast3_50_start_end = pangenie_DEL_sample_new_atleast3_50_zero[['CHROM', 'START_POS', 'END_POS', 'INFO_ID']]
pangenie_DEL_sample_new_atleast3_50_start_end.shape

(12670, 4)

#### insertions

In [16]:
pangenie_INS_sample_new = pangenie_INS_sample[[
 'CHROM','FILTER', 'INFO_ID', 'FORMAT', 'HG01114','HG03065','HG01505','HG02492','HG00096','HG03732','HG01573','NA20509','HG02587','HG01596','NA19983','HG00864','HG02011','HG00733','HG03371','NA19036','NA20847','HG00171','NA18534','NA18939','HG03683','NA19240','HG03009','HG02018','HG00514','NA19650'
     ]]
##### removed NA12329

In [118]:
for col in pangenie_INS_sample_new.iloc[:, 4:]:
    pangenie_INS_sample_new[col] = pangenie_INS_sample_new[col].str[:3]

In [119]:
for col in pangenie_INS_sample_new.columns:
    pangenie_INS_sample_new.loc[pangenie_INS_sample_new[col].astype(str).str.startswith('0/0'), col] = '0'
    pangenie_INS_sample_new.loc[pangenie_INS_sample_new[col].astype(str).str.startswith('.:.'), col] = '0'
    pangenie_INS_sample_new.loc[pangenie_INS_sample_new[col].astype(str).str.startswith('0/1'), col] = '1'
    pangenie_INS_sample_new.loc[pangenie_INS_sample_new[col].astype(str).str.startswith('1/0'), col] = '1'
    pangenie_INS_sample_new.loc[pangenie_INS_sample_new[col].astype(str).str.startswith('1/1'), col] = '1'

In [120]:
pangenie_INS_sample_new.iloc[:,4: ] = pangenie_INS_sample_new.iloc[:,4: ].astype(str).astype(int)
pangenie_INS_sample_new['Sum']=pangenie_INS_sample_new.iloc[:,4: ].sum(axis=1)
pangenie_INS_sample_new_atleast3 = pangenie_INS_sample_new[pangenie_INS_sample_new['Sum'].astype(int) >= 5]
pangenie_INS_sample_new_atleast3['START_POS'] = pangenie_INS_sample_new_atleast3['INFO_ID'].str.split('-', expand=True)[1]
pangenie_INS_sample_new_atleast3['TYPE'] = pangenie_INS_sample_new_atleast3['INFO_ID'].str.split('-', expand=True)[2]
pangenie_INS_sample_new_atleast3['LEN'] = pangenie_INS_sample_new_atleast3['INFO_ID'].str.split('-', expand=True)[3]

In [121]:
pangenie_INS_sample_new_atleast3.iloc[:,[31,33]] = pangenie_INS_sample_new_atleast3.iloc[:,[31,33]].astype(str).astype(int)
pangenie_INS_sample_new_atleast3['END_POS'] = pangenie_INS_sample_new_atleast3['START_POS']
pangenie_INS_sample_new_atleast3_new = pangenie_INS_sample_new_atleast3[[
'CHROM', 'START_POS','END_POS', 'TYPE','LEN', 'FILTER', 'INFO_ID', 'FORMAT', 'HG01114','HG03065','HG01505','HG02492','HG00096','HG03732','HG01573','NA20509','HG02587','HG01596','NA19983','HG00864','HG02011','HG00733','HG03371','NA19036','NA20847','HG00171','NA18534','NA18939','HG03683','NA19240','HG03009','HG02018','HG00514','NA19650', 'Sum'
    ]]

In [21]:
pangenie_INS_sample_new_atleast3_50 = pangenie_INS_sample_new_atleast3_new[pangenie_INS_sample_new_atleast3_new['LEN'] >= 50]
pangenie_INS_sample_new_atleast3_50.shape

(19992, 35)

In [22]:
pangenie_INS_sample_new_atleast3_50_zero = pangenie_INS_sample_new_atleast3_50[pangenie_INS_sample_new_atleast3_50.iloc[:, 8:].eq(0).any(1)]
pangenie_INS_sample_new_atleast3_50_start_end = pangenie_INS_sample_new_atleast3_50_zero[['CHROM', 'START_POS', 'END_POS', 'INFO_ID']]
pangenie_INS_sample_new_atleast3_50_start_end.shape

(17279, 4)

## SVs' impact on boundary strength

In [24]:
all_DEL_merged = pd.read_csv('all_DEL_26merged_cutoff_flank_inter30_5kb_boundaries_100kb_only01.bed', sep='\t', header=None)
all_DEL_merged.columns =['CHROM', 'START_POS', 'END_POS', 'INFO_ID', 'BOUND_CHR', 'FLANK_START', 'FLANK_END', 'BOUND_SCORE']

In [25]:
all_DEL_merged.shape

(12882, 8)

In [26]:
all_INS_merged = pd.read_csv('all_INS_26merged_cutoff_flank_inter30_5kb_boundaries_100kb_only01_sum5.bed', sep='\t', header=None)
all_INS_merged.columns =['CHROM', 'START_POS', 'END_POS', 'INFO_ID', 'BOUND_CHR', 'FLANK_START', 'FLANK_END', 'BOUND_SCORE']

In [27]:
all_INS_merged.shape

(17517, 8)

In [28]:
all_DEL_boundary_1 = all_DEL_merged.loc[all_DEL_merged.iloc[:, 4] != '.']
all_DEL_boundary_2 = all_DEL_boundary_1.reset_index(drop=True)

In [29]:
all_DEL_boundary_2.shape

(4047, 8)

In [30]:
all_INS_boundary_1 = all_INS_merged.loc[all_INS_merged.iloc[:, 4] != '.']
all_INS_boundary_2 = all_INS_boundary_1.reset_index(drop=True)

In [31]:
all_INS_boundary_2.shape

(5512, 8)

In [32]:
sample_ids = ['GM18534','GM18939','GM19036','GM19240','GM19650','GM19833','GM20509','GM20847','HG00096','HG00171',
              'HG00514','HG00733','HG00864','HG01114','HG01505','HG01573','HG01596','HG02011','HG02018','HG02492',
              'HG02587','HG03009','HG03065','HG03371','HG03683','HG03732']
file_directory = '/home/tun53987/Hi-C'
import glob
for i in sample_ids:
    file_pattern = os.path.join(file_directory, f"experiment_{i}/aligned/fanc_insulation_SCALE/BS_cutoff_inter30_10kb.boundaries_100kb.bed")
    sample_file = glob.glob(file_pattern)
    sample_merged_BS = pd.read_csv(sample_file[0], sep='\t', header=None)
    #sample_merged_BS = pd.read_csv(sample_file[0], sep='\t', header=None)
    sample_merged_BS =sample_merged_BS[[0, 1, 2, 3, 4, 5, 6, 8]]
    sample_merged_BS.columns =['BOUND_CHR', 'FLANK_START', 'FLANK_END', 'BOUND_SCORE','BOUND_CHR_samp', 'BOUND_START_samp', 'BOUND_END_samp', 'BOUND_SCORE_samp']
    sample_merged_BS_1 = sample_merged_BS.groupby(['BOUND_CHR', 'FLANK_START', 'FLANK_END'], as_index=False)['BOUND_SCORE_samp'].median()
    all_DEL_boundary_2[i] = all_DEL_boundary_2.merge(sample_merged_BS_1, how="left", left_on=["BOUND_CHR", "FLANK_START", "FLANK_END"], right_on=["BOUND_CHR", "FLANK_START", "FLANK_END"])['BOUND_SCORE_samp']
    all_INS_boundary_2[i] = all_INS_boundary_2.merge(sample_merged_BS_1, how="left", left_on=["BOUND_CHR", "FLANK_START", "FLANK_END"], right_on=["BOUND_CHR", "FLANK_START", "FLANK_END"])['BOUND_SCORE_samp']

#### Mann-Whitney U test for genotypes and boundary BS

In [35]:
all_DEL_boundary_2.shape

(4047, 34)

In [36]:
all_INS_boundary_2.shape

(5512, 34)

In [37]:
all_DEL_boundary_2['num_NaNs'] = all_DEL_boundary_2.iloc[:, 8:].isna().sum(1)
all_INS_boundary_2['num_NaNs'] = all_INS_boundary_2.iloc[:, 8:].isna().sum(1)

In [38]:
all_DEL_boundary_2['num_NaNs'].max()

26

In [39]:
all_INS_boundary_2['num_NaNs'].max()

26

In [40]:
all_DEL_boundary_2[all_DEL_boundary_2['num_NaNs'] == 26].index

Int64Index([  36,  177,  589,  590,  712,  729,  730,  809,  810,  909,  911,
            1028, 1041, 1226, 1272, 1273, 1302, 1307, 1308, 1324, 1480, 1634,
            1635, 1744, 1745, 1746, 1797, 1963, 2000, 2001, 2306, 2405, 2406,
            2407, 2408, 2644, 3121, 3245, 3369, 3423, 3424, 3593, 3609, 3755,
            3824, 3840, 3850, 4004],
           dtype='int64')

In [41]:
all_INS_boundary_2[all_INS_boundary_2['num_NaNs'] == 26].index

Int64Index([  34,  257,  258,  262,  784,  785,  851,  852, 1051, 1056, 1057,
            1058, 1171, 1183, 1205, 1206, 1302, 1316, 1502, 1533, 1801, 1802,
            1832, 1833, 2085, 2086, 2087, 2100, 2472, 2473, 2474, 2522, 2781,
            2822, 2838, 2839, 3212, 3213, 3318, 3319, 3781, 4221, 4238, 4331,
            4374, 4410, 4462, 4545, 4615, 4616, 4634, 4635, 5099, 5289, 5332,
            5422],
           dtype='int64')

In [42]:
all_DEL_boundary_2_noNA = all_DEL_boundary_2[all_DEL_boundary_2.num_NaNs < 26]
all_INS_boundary_2_noNA = all_INS_boundary_2[all_INS_boundary_2.num_NaNs < 26]

In [43]:
# Replace NaN Values with Zeros, because those NA boundary scores mean there is no boundary, so we can changed them to zero
all_DEL_boundary_3 = all_DEL_boundary_2_noNA.fillna(0).drop_duplicates()
all_INS_boundary_3 = all_INS_boundary_2_noNA.fillna(0).drop_duplicates()


In [44]:
all_DEL_boundary_3 = all_DEL_boundary_3.reset_index(drop=True)
all_INS_boundary_3 = all_INS_boundary_3.reset_index(drop=True)

In [45]:
all_DEL_boundary_3.shape

(3999, 35)

In [46]:
all_INS_boundary_3.shape

(5456, 35)

In [47]:
all_DEL_boundary_id = list(all_DEL_boundary_3['INFO_ID'])
DEL_list = list(set(all_DEL_boundary_id))
len(DEL_list)

3787

In [48]:
all_INS_boundary_id = list(all_INS_boundary_3['INFO_ID'])
INS_list = list(set(all_INS_boundary_id))
len(INS_list)

5221

In [50]:
DEL_genotype = pd.read_csv('pangenie_svs_26sample_ALL_DEL_atleast3_50_genotype_only01.bed', sep='\t', header=0)
INS_genotype = pd.read_csv('pangenie_svs_26sample_ALL_INS_atleast3_50_genotype_only01.bed', sep='\t', header=0)

In [51]:
DEL_genotype.shape

(12670, 35)

In [52]:
DEL_genotype_1 = DEL_genotype.iloc[:, :-1]
INS_genotype_1 = INS_genotype.iloc[:, :-1]
DEL_boundary_genotype = DEL_genotype_1[DEL_genotype_1['INFO_ID'].isin(all_DEL_boundary_id)]
INS_boundary_genotype = INS_genotype_1[INS_genotype_1['INFO_ID'].isin(all_INS_boundary_id)]

In [53]:
DEL_boundary_genotype = DEL_boundary_genotype[[
'CHROM', 'START_POS', 'END_POS', 'TYPE', 'LEN', 'FILTER', 'INFO_ID','FORMAT','NA18534','NA18939','NA19036','NA19240','NA19650','NA19983','NA20509','NA20847','HG00096','HG00171','HG00514','HG00733','HG00864','HG01114','HG01505','HG01573','HG01596','HG02011','HG02018','HG02492','HG02587','HG03009','HG03065','HG03371','HG03683','HG03732'
    ]]
INS_boundary_genotype = INS_boundary_genotype[[
'CHROM', 'START_POS', 'END_POS', 'TYPE', 'LEN', 'FILTER', 'INFO_ID','FORMAT','NA18534','NA18939','NA19036','NA19240','NA19650','NA19983','NA20509','NA20847','HG00096','HG00171','HG00514','HG00733','HG00864','HG01114','HG01505','HG01573','HG01596','HG02011','HG02018','HG02492','HG02587','HG03009','HG03065','HG03371','HG03683','HG03732'
    ]]

In [54]:
DEL_boundary_genotype_1 = DEL_boundary_genotype.reset_index(drop=True)
DEL_boundary_genotype_1.shape

(3787, 34)

In [55]:
INS_boundary_genotype_1 = INS_boundary_genotype.reset_index(drop=True)
INS_boundary_genotype_1.shape

(5221, 34)

In [56]:
### test for genotypes 0 and 1 (binary genotypes, only 0/0 and others)
pvalue= []

for i in range(0, len(all_DEL_boundary_3)):
    DEL_boundary_genotype_all = DEL_boundary_genotype_1[DEL_boundary_genotype_1.loc[:,'INFO_ID'] == all_DEL_boundary_3.loc[i,'INFO_ID']].iloc[:,8:]
    genotypes_list = DEL_boundary_genotype_all.iloc[0].values.tolist()
    boundary_score_list = all_DEL_boundary_3.iloc[i,8:34].values.tolist()

    df = pd.DataFrame({'genotypes' : genotypes_list,
                   'boundary_score' : boundary_score_list})
    
    df_new = df.assign(BS=df.groupby('genotypes').cumcount()).pivot('BS','genotypes','boundary_score')
      
    try:
        U, p = mannwhitneyu(x=df_new[0].dropna().tolist(), y=df_new[1].dropna().tolist(), method="exact", alternative = 'two-sided')
        #print (p)
        
    except ValueError:
        p = 'NA'
        #print (p)
    
    pvalue.append(p)
    
all_DEL_boundary_3['pvalue_0_1'] = pvalue

In [57]:
### test for genotypes 0 and 1 (binary genotypes, only 0/0 and others)
pvalue= []

for i in range(0, len(all_INS_boundary_3)):
    INS_boundary_genotype_all = INS_boundary_genotype_1[INS_boundary_genotype_1.loc[:,'INFO_ID'] == all_INS_boundary_3.loc[i,'INFO_ID']].iloc[:,8:]
    genotypes_list = INS_boundary_genotype_all.iloc[0].values.tolist()
    boundary_score_list = all_INS_boundary_3.iloc[i,8:34].values.tolist()

    df = pd.DataFrame({'genotypes' : genotypes_list,
                   'boundary_score' : boundary_score_list})
    
    df_new = df.assign(BS=df.groupby('genotypes').cumcount()).pivot('BS','genotypes','boundary_score')
      
    try:
        U, p = mannwhitneyu(x=df_new[0].dropna().tolist(), y=df_new[1].dropna().tolist(), method="exact", alternative = 'two-sided')
        #print (p)
        
    except ValueError:
        p = 'NA'
        #print (p)
    
    pvalue.append(p)
    
all_INS_boundary_3['pvalue_0_1'] = pvalue

In [58]:
all_DEL_boundary_3_sig = all_DEL_boundary_3[all_DEL_boundary_3['pvalue_0_1'] < 0.05]
all_DEL_boundary_3_sig.shape

(166, 36)

In [59]:
len(all_DEL_boundary_3_sig['INFO_ID'].drop_duplicates())

159

In [60]:
all_INS_boundary_3_sig = all_INS_boundary_3[all_INS_boundary_3['pvalue_0_1'] < 0.05]
all_INS_boundary_3_sig.shape

(274, 36)

In [61]:
len(all_INS_boundary_3_sig['INFO_ID'].drop_duplicates())

271

## SVs that significantly disrupt the TAD boundaries are also eQTLs

In [122]:
eqtl = pd.read_csv('qtl_results_all_v4_fdr0.05.txt', sep='\t')

In [63]:
eqtl.shape

(850482, 29)

In [64]:
pangenie_svs_sample_DEL_eqtl = eqtl[(eqtl['snp_id'].isin(all_DEL_boundary_3_sig['INFO_ID']))]

In [65]:
pangenie_svs_sample_DEL_eqtl.shape

(22, 29)

In [66]:
len(pangenie_svs_sample_DEL_eqtl['snp_id'].drop_duplicates())

17

In [67]:
len(pangenie_svs_sample_DEL_eqtl['feature_id'].drop_duplicates())

20

In [68]:
pangenie_svs_sample_INS_eqtl = eqtl[(eqtl['snp_id'].isin(all_INS_boundary_3_sig['INFO_ID']))]

In [69]:
pangenie_svs_sample_INS_eqtl.shape

(39, 29)

In [70]:
len(pangenie_svs_sample_INS_eqtl['snp_id'].drop_duplicates())

22

In [71]:
len(pangenie_svs_sample_INS_eqtl['feature_id'].drop_duplicates())

37

In [72]:
eqtl_DEL_boundary_gene = list(pangenie_svs_sample_DEL_eqtl['feature_id'])
len(eqtl_DEL_boundary_gene)

22

In [73]:
eqtl_INS_boundary_gene = list(pangenie_svs_sample_INS_eqtl['feature_id'])
len(eqtl_INS_boundary_gene)

39

In [74]:
gene_expression = pd.read_csv('featureCounts_v2.genes.counts.edgeR.log.txt', sep='\t', header=0)
gene_expression_DEL_boundary = gene_expression.filter(items = eqtl_DEL_boundary_gene, axis=0)
gene_expression_INS_boundary = gene_expression.filter(items = eqtl_INS_boundary_gene, axis=0)

In [75]:
sample = ['HG01114','HG03065','HG01505','HG02492','HG00096','HG03732','HG01573','NA20509','HG02587','HG01596','NA19983','HG00864','HG02011','HG00733','HG03371','NA19036','NA20847','HG00171','NA18534','NA18939','HG03683','NA19240','HG03009','HG02018','HG00514','NA19650']
gene_expression_DEL_boundary_sample = gene_expression_DEL_boundary.filter(regex='|'.join(sample), axis=1)
gene_expression_INS_boundary_sample = gene_expression_INS_boundary.filter(regex='|'.join(sample), axis=1)
## only keep the first 7 characters of the columns name
gene_DEL_boundary_sample_rename = gene_expression_DEL_boundary_sample.rename(columns = lambda x : str(x)[:7])
gene_INS_boundary_sample_rename = gene_expression_INS_boundary_sample.rename(columns = lambda x : str(x)[:7])
gene_DEL_boundary_sample_rename_new =  gene_DEL_boundary_sample_rename.reset_index().rename(columns={'index': 'gene'})
gene_INS_boundary_sample_rename_new =  gene_INS_boundary_sample_rename.reset_index().rename(columns={'index': 'gene'})

In [76]:
INFO_ID = list(pangenie_svs_sample_DEL_eqtl.iloc[:, 1])
len(INFO_ID)

22

In [77]:
gene_DEL_boundary_sample_rename_new['INFO_ID'] = INFO_ID 
gene_DEL_boundary_sample_rename_new = gene_DEL_boundary_sample_rename_new[[
'gene','INFO_ID', 'NA18534','NA18939','NA19036','NA19240','NA19650','NA19983','NA20509','NA20847','HG00096','HG00171','HG00514','HG00733','HG00864','HG01114','HG01505','HG01573','HG01596','HG02011','HG02018','HG02492','HG02587','HG03009','HG03065','HG03371','HG03683','HG03732'
    ]]

In [78]:
gene_DEL_boundary_sample_rename_new.shape

(22, 28)

In [79]:
INFO_ID = list(pangenie_svs_sample_INS_eqtl.iloc[:, 1])
len(INFO_ID)

39

In [80]:
gene_INS_boundary_sample_rename_new['INFO_ID'] = INFO_ID 
gene_INS_boundary_sample_rename_new = gene_INS_boundary_sample_rename_new[[
'gene','INFO_ID', 'NA18534','NA18939','NA19036','NA19240','NA19650','NA19983','NA20509','NA20847','HG00096','HG00171','HG00514','HG00733','HG00864','HG01114','HG01505','HG01573','HG01596','HG02011','HG02018','HG02492','HG02587','HG03009','HG03065','HG03371','HG03683','HG03732'
    ]]

In [81]:
gene_INS_boundary_sample_rename_new.shape

(39, 28)

In [82]:
### test for genotypes 0 and 1
pvalue= []

for i in range(0, len(gene_DEL_boundary_sample_rename_new)):
    DEL_boundary_genotype_all = DEL_boundary_genotype[DEL_boundary_genotype.loc[:,'INFO_ID'] == gene_DEL_boundary_sample_rename_new.loc[i,'INFO_ID']].iloc[:,8:35]
    genotypes_list = DEL_boundary_genotype_all.iloc[0].values.tolist()
    gene_expression_list = gene_DEL_boundary_sample_rename_new.iloc[i, 2:28].values.tolist()

    df = pd.DataFrame({'genotypes' : genotypes_list,
                   'gene_expression' : gene_expression_list})
    
    df_new = df.assign(expression=df.groupby('genotypes').cumcount()).pivot('expression','genotypes','gene_expression')
    
    try:
        U, p = mannwhitneyu(x=df_new[0].dropna().tolist(), y=df_new[1].dropna().tolist(), method="exact", alternative = 'two-sided')
        #print (p)
        
    except KeyError:
        p = 'NA'
        #print (p)
    
    pvalue.append(p)
    
gene_DEL_boundary_sample_rename_new['pvalue_0_1'] = pvalue

In [83]:
gene_DEL_boundary_sample_rename_new[gene_DEL_boundary_sample_rename_new['pvalue_0_1'] < 0.05]

,gene,INFO_ID,NA18534,NA18939,NA19036,NA19240,NA19650,NA19983,NA20509,NA20847,...,HG02011,HG02018,HG02492,HG02587,HG03009,HG03065,HG03371,HG03683,HG03732,pvalue_0_1
4,XKR9,chr8-70671321-DEL-1088,0.230725,0.078901,0.301779,0.267848,0.270861,0.136379,0.530737,0.218365,...,0.446265,0.214502,0.225182,0.273766,0.139141,0.287291,0.175466,0.463090,0.130936,0.000040
5,ERICH1,chr8-639689-DEL-149,1.748027,2.397384,1.672566,2.079122,1.959391,1.973457,2.233838,2.596048,...,1.900657,1.929456,1.850300,1.798068,2.061475,1.680383,1.861537,2.319171,2.234540,0.027393
6,ERICH1,chr8-644401-DEL-5014,1.748027,2.397384,1.672566,2.079122,1.959391,1.973457,2.233838,2.596048,...,1.900657,1.929456,1.850300,1.798068,2.061475,1.680383,1.861537,2.319171,2.234540,0.000122
14,LINC00638,chr14-104826838-DEL-186,0.609886,1.014848,0.708984,0.473654,0.914227,0.959242,0.875473,1.154632,...,0.631733,0.610735,0.745175,0.711802,0.736278,0.848344,0.906329,1.183598,1.091448,0.000173
16,COG4,chr16-70519107-DEL-320,4.491429,4.282323,4.564743,4.116230,4.709587,4.460420,4.604825,4.606563,...,4.492422,4.366832,4.954317,4.530447,4.446040,4.705948,4.383704,4.411063,4.815297,0.034205


In [84]:
### test for genotypes 0 and 1
pvalue= []

for i in range(0, len(gene_INS_boundary_sample_rename_new)):
    INS_boundary_genotype_all = INS_boundary_genotype[INS_boundary_genotype.loc[:,'INFO_ID'] == gene_INS_boundary_sample_rename_new.loc[i,'INFO_ID']].iloc[:,8:35]
    genotypes_list = INS_boundary_genotype_all.iloc[0].values.tolist()
    gene_expression_list = gene_INS_boundary_sample_rename_new.iloc[i, 2:28].values.tolist()

    df = pd.DataFrame({'genotypes' : genotypes_list,
                   'gene_expression' : gene_expression_list})
    
    df_new = df.assign(expression=df.groupby('genotypes').cumcount()).pivot('expression','genotypes','gene_expression')
    
    try:
        U, p = mannwhitneyu(x=df_new[0].dropna().tolist(), y=df_new[1].dropna().tolist(), method="exact", alternative = 'two-sided')
        #print (p)
        
    except KeyError:
        p = 'NA'
        #print (p)
    
    pvalue.append(p)
    
gene_INS_boundary_sample_rename_new['pvalue_0_1'] = pvalue

In [85]:
gene_INS_boundary_sample_rename_new[gene_INS_boundary_sample_rename_new['pvalue_0_1'] < 0.05]

,gene,INFO_ID,NA18534,NA18939,NA19036,NA19240,NA19650,NA19983,NA20509,NA20847,...,HG02011,HG02018,HG02492,HG02587,HG03009,HG03065,HG03371,HG03683,HG03732,pvalue_0_1
0,CCDC163,chr1-45497763-INS-354,2.451951,2.138003,2.807306,1.613811,2.671173,2.302246,2.088805,2.481762,...,2.629090,2.677383,2.596948,2.678364,2.705544,1.964997,2.047240,2.476094,2.718013,0.012772
1,TESK2,chr1-45497763-INS-354,3.727556,3.232982,3.776108,2.972572,2.959789,2.984399,3.070232,3.653215,...,3.431179,3.117390,3.183825,3.841491,3.130307,2.863411,3.244661,3.323535,3.269226,0.041446
2,MARC2,chr1-220781004-INS-321,3.142149,1.557398,1.114918,0.850355,1.038078,3.224358,2.717753,2.494286,...,1.595943,1.738261,1.704518,3.030049,1.363934,2.805114,2.416636,1.845741,1.567951,0.000230
3,C1orf115,chr1-220781004-INS-321,3.670019,1.288197,1.381543,1.476605,0.756454,3.365972,3.003896,2.257705,...,1.765845,2.393801,0.850476,3.583112,1.447648,1.653860,3.991472,1.556729,1.269378,0.000062
15,LOC112267974,chr6-130293639-INS-295,0.695876,1.733167,1.656363,3.240195,1.345622,1.799491,1.392224,2.942051,...,2.221368,1.429414,1.516744,2.910385,2.157659,1.750333,1.691537,0.901506,2.504318,0.046354
16,TMEM200A,chr6-130293639-INS-295,2.303405,2.650858,3.043825,3.450595,2.748188,2.839752,2.489334,3.862035,...,3.410804,2.712212,2.924135,4.531084,3.361059,3.611612,3.634799,1.505742,3.210732,0.007681
18,SAMD3,chr6-130293639-INS-295,0.224783,0.458532,0.441411,1.106813,0.270949,0.730064,0.337769,1.007796,...,0.713667,0.377570,0.623951,1.134473,0.641602,0.516529,0.569114,0.264691,0.755668,0.023364
20,LOC102723656,chr7-55783655-INS-305,0.138756,0.021388,0.522665,0.309137,0.105346,0.367735,0.200672,0.375699,...,0.336785,0.300737,0.302621,0.808641,0.204893,1.023188,0.426118,0.371994,0.402534,0.003054
21,LOC101060341,chr7-55783655-INS-305,0.052978,0.055925,0.233678,0.153011,0.049746,0.235489,0.235998,0.188121,...,0.110397,0.057820,0.150939,0.418166,0.151934,0.511560,0.332467,0.419341,0.226926,0.000528
22,LCN8,chr9-136779140-INS-50,0.157390,0.000000,0.000000,0.000000,0.000000,0.000000,0.005477,0.020052,...,0.016479,0.011318,0.000000,0.000000,0.027785,0.023173,0.000000,0.262920,0.000000,0.000091


## SVs that significantly disrupt the TAD boundaries are also sQTLs

In [86]:
sqtl = pd.read_csv('qtl_results_all_v2_fdr0.05.txt', sep = '\t')

In [87]:
sqtl.shape

(1103872, 22)

In [88]:
pangenie_svs_sample_DEL_sqtl = sqtl[(sqtl['snp_id'].isin(all_DEL_boundary_3_sig['INFO_ID']))]

In [89]:
pangenie_svs_sample_DEL_sqtl.shape

(28, 22)

In [90]:
len(pangenie_svs_sample_DEL_sqtl['snp_id'].drop_duplicates())

10

In [91]:
len(pangenie_svs_sample_DEL_sqtl['feature_id'].drop_duplicates())

25

In [92]:
pangenie_svs_sample_INS_sqtl = sqtl[(sqtl['snp_id'].isin(all_INS_boundary_3_sig['INFO_ID']))]

In [93]:
pangenie_svs_sample_INS_sqtl.shape

(67, 22)

In [94]:
len(pangenie_svs_sample_INS_sqtl['snp_id'].drop_duplicates())

9

In [95]:
len(pangenie_svs_sample_INS_sqtl['feature_id'].drop_duplicates())

67

In [96]:
sqtl_DEL_boundary_feature = list(pangenie_svs_sample_DEL_sqtl['feature_id'])
sqtl_INS_boundary_feature = list(pangenie_svs_sample_INS_sqtl['feature_id'])

In [97]:
splicing_ratio = pd.read_csv('LCL_perind.counts.qqnorm.CovariatesRemoved.cs.txt', sep='\t', header = 0, index_col=0)

In [98]:
splicing_ratio.shape

(143713, 444)

In [99]:
splicing_DEL_boundary = splicing_ratio.filter(items = sqtl_DEL_boundary_feature, axis=0)
splicing_INS_boundary = splicing_ratio.filter(items = sqtl_INS_boundary_feature, axis=0)

In [100]:
sample = ['HG01114','HG03065','HG01505','HG02492','HG00096','HG03732','HG01573','NA20509','HG02587','HG01596','NA19983','HG00864','HG02011','HG00733','HG03371','NA19036','NA20847','HG00171','NA18534','NA18939','HG03683','NA19240','HG03009','HG02018','HG00514','NA19650']
splicing_DEL_boundary_sample = splicing_DEL_boundary.filter(regex='|'.join(sample), axis=1)
splicing_INS_boundary_sample = splicing_INS_boundary.filter(regex='|'.join(sample), axis=1)
## only keep the first 7 characters of the columns name
splicing_DEL_boundary_sample_rename = splicing_DEL_boundary_sample.rename(columns = lambda x : str(x)[:7])
splicing_INS_boundary_sample_rename = splicing_INS_boundary_sample.rename(columns = lambda x : str(x)[:7])

In [101]:
splicing_DEL_boundary_sample_rename_new = splicing_DEL_boundary_sample_rename.reset_index().rename(columns={'index': 'feature_id'})
splicing_INS_boundary_sample_rename_new = splicing_INS_boundary_sample_rename.reset_index().rename(columns={'index': 'feature_id'})

In [102]:
INFO_ID = list(pangenie_svs_sample_DEL_sqtl.iloc[:, 1])
len(INFO_ID)

28

In [103]:
splicing_DEL_boundary_sample_rename_new['INFO_ID'] = INFO_ID 
splicing_DEL_boundary_sample_rename_new = splicing_DEL_boundary_sample_rename_new[[
'feature_id','INFO_ID', 'NA18534','NA18939','NA19036','NA19240','NA19650','NA19983','NA20509','NA20847','HG00096','HG00171','HG00514','HG00733','HG00864','HG01114','HG01505','HG01573','HG01596','HG02011','HG02018','HG02492','HG02587','HG03009','HG03065','HG03371','HG03683','HG03732'
    ]]

In [104]:
INFO_ID = list(pangenie_svs_sample_INS_sqtl.iloc[:, 1])
len(INFO_ID)

67

In [105]:
splicing_INS_boundary_sample_rename_new['INFO_ID'] = INFO_ID 
splicing_INS_boundary_sample_rename_new = splicing_INS_boundary_sample_rename_new[[
'feature_id','INFO_ID', 'NA18534','NA18939','NA19036','NA19240','NA19650','NA19983','NA20509','NA20847','HG00096','HG00171','HG00514','HG00733','HG00864','HG01114','HG01505','HG01573','HG01596','HG02011','HG02018','HG02492','HG02587','HG03009','HG03065','HG03371','HG03683','HG03732'
    ]]

In [106]:
### test for genotypes 0 and 1
pvalue= []
for i in range(0, len(splicing_DEL_boundary_sample_rename_new)):
    DEL_boundary_genotype_all = DEL_boundary_genotype[DEL_boundary_genotype.loc[:,'INFO_ID'] == splicing_DEL_boundary_sample_rename_new.loc[i,'INFO_ID']].iloc[:,8:35]
    genotypes_list = DEL_boundary_genotype_all.iloc[0].values.tolist()
    splicing_ratio_list = splicing_DEL_boundary_sample_rename_new.iloc[i, 2:28].values.tolist()

    df = pd.DataFrame({'genotypes' : genotypes_list,
                   'splicing_ratio' : splicing_ratio_list})
    
    df_new = df.assign(splicing=df.groupby('genotypes').cumcount()).pivot('splicing','genotypes','splicing_ratio')
    
    try:
        U, p = mannwhitneyu(x=df_new[0].dropna().tolist(), y=df_new[1].dropna().tolist(), method="exact", alternative = 'two-sided')
        #print (p)
        
    except KeyError:
        p = 'NA'
        #print (p)
    
    pvalue.append(p)
    
splicing_DEL_boundary_sample_rename_new['pvalue_0_1'] = pvalue

In [107]:
splicing_DEL_boundary_sample_rename_new[splicing_DEL_boundary_sample_rename_new['pvalue_0_1'] < 0.05]

,feature_id,INFO_ID,NA18534,NA18939,NA19036,NA19240,NA19650,NA19983,NA20509,NA20847,...,HG02011,HG02018,HG02492,HG02587,HG03009,HG03065,HG03371,HG03683,HG03732,pvalue_0_1
1,6:87391835:87392725:clu_24429_+,chr6-87408456-DEL-133,-0.936184,0.012348,1.076310,2.158304,-0.932265,0.412867,-0.841126,-0.767154,...,-0.708359,0.036234,-1.040756,-1.141381,0.385166,-0.996222,-1.159694,0.681233,-0.554325,0.014847
4,8:70669538:70674818:clu_22301_+,chr8-70671321-DEL-1088,-0.005980,-0.950419,-0.399550,-0.929802,2.013419,-1.227758,2.069432,-1.077835,...,-0.780188,-1.215290,-0.646946,1.773509,-1.187269,0.009941,1.334757,0.612355,-0.937157,0.000244
5,8:664676:668598:clu_8652_-,chr8-644401-DEL-5014,1.245457,-0.414052,1.217779,-0.759694,1.209294,1.131286,-0.186955,0.427311,...,0.134623,0.345217,1.370163,1.423127,0.506337,1.393303,0.331152,0.336981,0.661554,0.012405
6,8:640834:668598:clu_8652_-,chr8-639689-DEL-149,-1.331250,1.221957,-1.475604,0.898362,-1.557109,-1.498409,0.061330,0.288855,...,0.414936,0.447919,-1.577237,-1.630796,-0.181406,-1.552799,-0.199111,-0.010256,-0.255825,0.000301
7,8:664676:668598:clu_8652_-,chr8-639689-DEL-149,1.245457,-0.414052,1.217779,-0.759694,1.209294,1.131286,-0.186955,0.427311,...,0.134623,0.345217,1.370163,1.423127,0.506337,1.393303,0.331152,0.336981,0.661554,0.001621
8,8:692313:692478:clu_8654_-,chr8-639689-DEL-149,-0.853231,1.101585,-0.215530,-0.143407,-1.490113,0.476855,0.299725,-0.165151,...,0.064356,1.951518,-0.711567,-0.632646,-0.964003,-0.240850,-0.123726,0.171931,0.935975,0.006448
9,8:70652752:70653695:clu_22301_+,chr8-70671321-DEL-1088,0.704935,1.305387,0.883035,0.859624,-1.808690,0.580523,-2.483752,0.948607,...,-0.185246,0.328117,0.560184,-1.196977,0.807123,-0.060676,-0.594006,0.057426,0.818711,0.000140
10,8:640834:668598:clu_8652_-,chr8-644401-DEL-5014,-1.331250,1.221957,-1.475604,0.898362,-1.557109,-1.498409,0.061330,0.288855,...,0.414936,0.447919,-1.577237,-1.630796,-0.181406,-1.552799,-0.199111,-0.010256,-0.255825,0.040924
23,20:35632923:35664760:clu_21372_-,chr20-35960724-DEL-60,1.746169,0.529841,0.298889,-0.216541,-0.391229,0.191113,0.014453,-0.368097,...,1.366691,-0.306092,-0.028003,-0.167166,-0.169622,-0.432385,-0.916697,-0.475933,-0.963485,0.020419
24,20:35655344:35658930:clu_21372_-,chr20-35960724-DEL-60,-2.666742,-0.113100,-0.047915,0.324691,0.682168,0.111165,0.232633,0.199351,...,-0.019236,0.522568,0.629328,0.722698,0.518111,0.397488,0.963883,0.527364,1.150455,0.031556


In [108]:
len(splicing_DEL_boundary_sample_rename_new[splicing_DEL_boundary_sample_rename_new['pvalue_0_1'] < 0.05]['INFO_ID'].drop_duplicates())

5

In [109]:
len(splicing_DEL_boundary_sample_rename_new[splicing_DEL_boundary_sample_rename_new['pvalue_0_1'] < 0.05]['feature_id'].drop_duplicates())

11

In [110]:
### test for genotypes 0 and 1
pvalue= []
for i in range(0, len(splicing_INS_boundary_sample_rename_new)):
    INS_boundary_genotype_all = INS_boundary_genotype[INS_boundary_genotype.loc[:,'INFO_ID'] == splicing_INS_boundary_sample_rename_new.loc[i,'INFO_ID']].iloc[:,8:35]
    genotypes_list = INS_boundary_genotype_all.iloc[0].values.tolist()
    splicing_ratio_list = splicing_INS_boundary_sample_rename_new.iloc[i, 2:28].values.tolist()

    df = pd.DataFrame({'genotypes' : genotypes_list,
                   'splicing_ratio' : splicing_ratio_list})
    
    df_new = df.assign(splicing=df.groupby('genotypes').cumcount()).pivot('splicing','genotypes','splicing_ratio')
    
    try:
        U, p = mannwhitneyu(x=df_new[0].dropna().tolist(), y=df_new[1].dropna().tolist(), method="exact", alternative = 'two-sided')
        #print (p)
        
    except KeyError:
        p = 'NA'
        #print (p)
    
    pvalue.append(p)
    
splicing_INS_boundary_sample_rename_new['pvalue_0_1'] = pvalue

In [111]:
splicing_INS_boundary_sample_rename_new[splicing_INS_boundary_sample_rename_new['pvalue_0_1'] < 0.05]

,feature_id,INFO_ID,NA18534,NA18939,NA19036,NA19240,NA19650,NA19983,NA20509,NA20847,...,HG02011,HG02018,HG02492,HG02587,HG03009,HG03065,HG03371,HG03683,HG03732,pvalue_0_1
2,1:45495166:45496556:clu_28892_-,chr1-45497763-INS-354,-0.097358,0.798389,0.295351,-0.274953,0.299073,0.053754,-0.131064,-0.416363,...,-0.276865,0.627438,-0.162830,-1.278557,-0.814071,0.670716,0.499751,-1.262515,0.616364,0.031556
4,1:45551155:45552436:clu_13870_+,chr1-45497763-INS-354,-1.746480,0.475494,-0.078211,-0.445241,-0.698239,-2.578916,0.338158,1.013390,...,0.299322,-0.391775,0.861487,0.718653,-0.177377,-0.359558,-0.607451,0.442949,-0.515916,0.000822
9,4:124673:126848:clu_40368_+,chr4-152957-INS-5083,-0.185285,-1.457131,1.114331,0.885232,0.205516,-1.222210,0.784782,-0.106020,...,0.081332,-0.704891,1.289569,0.987919,0.815414,0.818722,-2.221038,1.112181,-1.826948,0.027859
11,4:124673:130788:clu_40368_+,chr4-152957-INS-5083,0.635271,1.789222,-1.442438,-1.223004,-0.283380,1.655514,-1.493125,1.036192,...,0.066262,0.952385,-1.504064,-1.546094,-1.535493,-1.452881,1.895520,-1.420374,1.707532,0.039065
17,4:125296:130788:clu_40368_+,chr4-152957-INS-5083,1.585531,1.134742,-0.938089,-0.818587,-0.346082,0.789514,-0.971336,0.510938,...,0.473616,0.600961,-0.951855,-0.996012,-0.997648,-0.927929,0.839999,-0.965777,1.664260,0.016010
37,6:130385236:130439348:clu_24586_+,chr6-130293639-INS-295,0.267463,0.252313,0.004427,-0.021482,0.139053,-0.071580,-0.063894,-0.183647,...,-0.018760,0.259926,-0.221718,-0.018741,0.209585,0.049197,-0.006632,0.072590,-0.163549,0.010767
51,7:56021231:56031929:clu_36747_-,chr7-55783655-INS-305,0.157423,0.861163,-1.027186,-0.699761,0.890527,-0.395204,-0.119004,-0.177994,...,0.034311,-0.126273,-0.288276,-0.671426,-0.003130,-1.038168,-0.392324,-0.345987,-0.006782,0.000528
52,7:56021231:56116015:clu_36747_-,chr7-55783655-INS-305,0.156402,-1.139293,0.946572,0.778528,-1.158309,0.544943,0.327408,0.349796,...,0.216117,0.292999,0.477960,0.826821,0.199462,0.987147,0.612970,0.498881,0.228091,0.000528
53,7:56011869:56021073:clu_36746_-,chr7-55783655-INS-305,0.113327,-0.572458,0.596230,-0.203613,-0.600021,0.536893,-0.028733,-0.189171,...,-0.316349,0.395567,-0.023145,0.523478,-0.303071,0.860438,0.487270,0.590745,-0.057283,0.023364
55,15:89321012:89321736:clu_2903_-,chr15-89314067-INS-62,-0.028732,-0.452393,0.357381,-0.294620,-0.962535,0.208674,-0.972501,-0.187520,...,-0.599404,-0.639060,1.689366,-0.471705,1.179762,-0.328591,-0.546996,-0.012979,-1.772711,0.021407


In [112]:
len(splicing_INS_boundary_sample_rename_new[splicing_INS_boundary_sample_rename_new['pvalue_0_1'] < 0.05]['INFO_ID'].drop_duplicates())

6

In [113]:
len(splicing_INS_boundary_sample_rename_new[splicing_INS_boundary_sample_rename_new['pvalue_0_1'] < 0.05]['feature_id'].drop_duplicates())

15